# 흑백요리사 식당 500m 격자 생성 및 시각화

이 노트북은 `캐치테이블_가게정보.csv` 파일의 위도/경도 정보를 활용하여 다음 작업을 수행합니다.
1. **좌표계 변환**: WGS84(위경도) -> EPSG:5179(미터 좌표계)
2. **격자 생성**: 각 식당 기준 반경 500m를 커버하는 정사각형 격자 생성 (가로세로 1km)
3. **시각화**: Folium을 이용한 지도 시각화

In [2]:
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point
import os

In [3]:
# 데이터 파일 경로
file_path = r"c:\Users\USER\Documents\웅진씽크빅kdt\흑백요리사\캐치테이블_가게정보.csv"

# CSV 읽기
df = pd.read_csv(file_path)
print(f"✅ 데이터 로드 완료: {len(df)}개 식당")
df[['restaurant', 'lat', 'lon']].head()

✅ 데이터 로드 완료: 118개 식당


,restaurant,lat,lon
0,유용욱 바베큐 연구소,37.543957,126.972782
1,IMOK Smoke Dining,37.521126,127.019125
2,라망시크레,37.559666,126.979542
3,Original Numbers 청담,37.525096,127.040738
4,앰배서더 서울 풀만 호빈,37.560592,127.002094


In [ ]:
def create_500m_grid_from_locations(df, lat_col, lon_col):
    """
    위경도 데이터를 받아 500m 반경 기반 격자(EPSG:5179)를 생성하는 함수
    """
    # 1. GeoDataFrame 생성 (EPSG:4326)
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
    
    # 2. 좌표계 변환 (EPSG:5179)
    gdf_proj = gdf.to_crs(epsg=5179)
    
    # 3. 500m 버퍼 -> 사각형(Envelope) 변환
    # 반경 500m이므로 지름은 1000m (1km)가 됩니다.
    gdf_proj['grid_geometry'] = gdf_proj.geometry.buffer(300).envelope
    
    # 4. grid_geometry를 메인으로 설정하고 정리
    gdf_grid = gdf_proj.set_geometry('grid_geometry')
    gdf_grid = gdf_grid.drop(columns=['geometry'], errors='ignore') # 기존 Point 컬럼 삭제
    gdf_grid = gdf_grid.rename_geometry('geometry')
    
    return gdf_grid

In [ ]:
# 함수 실행
try:
    gdf_result = create_500m_grid_from_locations(df, 'lat', 'lon')
    
    # 면적 계산 (검증)
    # 1000m * 1000m = 1,000,000 m2 이어야 함
    gdf_result['area_m2'] = gdf_result.geometry.area
    
    print("✅ 500m 반경(1km 격자) 생성 완료!")
    display(gdf_result[['restaurant', 'area_m2', 'geometry']].head())
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")

✅ 500m 반경(1km 격자) 생성 완료!


,restaurant,area_m2,geometry
0,유용욱 바베큐 연구소,360000.0,"POLYGON ((953126.305 1949233.645, 953726.305 1..."
1,IMOK Smoke Dining,360000.0,"POLYGON ((957207.298 1946678.729, 957807.298 1..."
2,라망시크레,360000.0,"POLYGON ((953733.127 1950973.172, 954333.127 1..."
3,Original Numbers 청담,360000.0,"POLYGON ((959119.257 1947109.589, 959719.257 1..."
4,앰배서더 서울 풀만 호빈,360000.0,"POLYGON ((955725.477 1951065.083, 956325.477 1..."


In [9]:
# 시각화를 위해 다시 위경도 좌표계(EPSG:4326)로 변환
gdf_viz = gdf_result.to_crs(epsg=4326)

# 중심점 계산
center_lat = gdf_viz.geometry.centroid.y.mean()
center_lon = gdf_viz.geometry.centroid.x.mean()

# 지도 생성 (줌 레벨 조정)
m = folium.Map(location=[center_lat, center_lon], zoom_start=11)

# 격자 추가
folium.GeoJson(
    gdf_viz,
    name='500m Grid',
    style_function=lambda x: {'fillColor': '#0000ff', 'color': 'blue', 'weight': 1, 'fillOpacity': 0.2},
    tooltip=folium.GeoJsonTooltip(fields=['restaurant', 'area_m2'], aliases=['식당명', '면적(m2)'])
).add_to(m)

# 지도 출력
m

C:\Users\USER\AppData\Local\Temp\ipykernel_9524\2382406333.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = gdf_viz.geometry.centroid.y.mean()
C:\Users\USER\AppData\Local\Temp\ipykernel_9524\2382406333.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = gdf_viz.geometry.centroid.x.mean()


In [ ]:
# 결과 저장
output_geojson = r"c:\Users\USER\Documents\웅진씽크빅kdt\흑백요리사\result_grid_500m.geojson"
gdf_result.to_file(output_geojson, driver='GeoJSON')
print(f"💾 파일 저장 완료: {output_geojson}")